## Non-Linear/Extended Kalman filter (EKF)

Ex: self-driving car position state estimation in order to compute the angle model measured by front-facing camera of the vehicle

![image](https://github.com/afondiel/Self-Driving-Cars-Specialization-Coursera/blob/main/Course2-State-Estimation-and-Localization-for-Self-Driving-Cars/resources/w2/l3-EKF8-short.png?raw=true)


In [1]:
import numpy as np
from numpy.linalg import inv
import matplotlib.pyplot as plt

# initialize the parameter and input estimates:
# x0 = [p, p_dot].T  => the estimate parameter vector => position of the vehicle (m)
# u0 = control signal => scalar
"""estimate to be corrected"""
# x0 : mean
xk = np.array([[0, 5]]).T
# P0 : Covariance
Pk = np.array([[0.01, 0], [0, 1]])

# u => acceleration of the vehicle / double derivative of position (p_ddot) 
# vehicle moving with constant speed => a = 0
uk  = -2.0 # [m/s^2]
 
# (S , D) are known
S, D = (20, 40)

# Measurement model 
# yk = phi = h(pk, vk) # [m]
################################
## phi = Arctan(S/D_pk) + vk  ##
################################
xk_p = xk
meas_model = np.arctan(S/((D - xk_p)**2 + S**2)) # phi in Rad
# first measurement
yk = np.pi/6 # rad
# measurement time at t=0.5s
dt = 0.5 # s

# Noise density
Rk = 0.01
Qk = 0.1*np.eye(2)

# Process/noise
wk = np.array([0, Qk])
# measurement noise
vk = np.array([0, 0.01])

# Motion model Jacobian 
Fk = np.array([[1, dt], [0, 1]], dtype=float)
Lk = np.eye(2)
Gk = np.array([[0, dt]]).T


# Measurement model Jacobian
# meas_model : array([[0.00999967],
#        [0.01230707]]) 
# format the value to 3 decimal numbers precision
Hk = np.array([[np.round(meas_model[0], 3), 0]], dtype=float)
Mk = np.eye(2)

"""  
- KF implementation
- Prediction  (xk_check, Pk_check)
- Optmal Gain (Kk)
- Correction  (xk_hat, Pk_hat)

  - Number of measurements
  - The measurement data may come from a real time sensor measurement of the vehicle ou from a batch of measurement (.csv file for ex.)
    - num_meas = read_sensor_ measurement(camera) ?
    - num_meas = batch_array ? 
    - x_hist = np.zeros((num_meas + 1, 2))
    - P_hist = np.zeros((num_meas + 1, 2, 2))
"""  
# for k in num_meas: # in case we have more than one measurement

# Prediction (xk_check, Pk_check)
## motion/process model
xk_p = np.dot(Fk, xk) + np.dot(Gk, uk)

## uncertainty
Pk_p = np.dot(Fk, np.dot(Pk, Fk.T)) + np.dot(Lk, np.dot(Qk, Lk.T))

print(f'xk_p:{xk_p}, Pk_p:{Pk_p}, Hk:{Hk}, Hk.T:{Hk.T}')
print(f'xk_p-.shape:{xk_p.shape}, Pk_p-.shape:{Pk_p.shape}, Hk-shape:{Hk.shape}, Hk.T-shape:{Hk.T.shape}')

## Optimal Gain (Kk)
# ValueError: shapes (2,1) and (2,2) not aligned: 1 (dim 1) != 2 (dim 0)
# solution I inverteded to respect dot product : dot(inv, Pk_p_dot_Hk) <=> (2x2)*(2x1)
Kk = np.dot( inv(np.dot(Hk, np.dot(Pk_p, Hk.T)) + np.dot(Mk, np.dot(Rk, Mk.T))), np.dot(Pk_p, Hk.T)) 

# Correction  (xk_hat, Pk_hat)
## measurement model
# print(f'Kk:{Kk}, Pk_p:{Pk_p}, Hk:{Hk}, Hk.T:{Hk.T},  Rk:{Rk}, yk:{yk}\n')
# print(f'Kk-.shape:{Kk.shape}, Pk_p-.shape:{Pk_p.shape}, Hk-shape:{Hk.shape}, Hk.T-shape:{Hk.T.shape}')
## xk corrected
xk = xk_p + np.dot(Kk, (yk - np.dot(Hk, xk_p))) 
print(f'xk:{xk}, xk-shape:{xk_p.shape}\n')

## Pk corrected
I = np.eye(2) # Identity matrix (2x2)
Pk = np.dot(I - np.dot(Kk, Hk), Pk_p) 

print(f'Pk:{Pk}, Pk_p:{Pk_p},  Pk-shape:{Pk.shape},  Pk_p-shape:{Pk_p.shape}')
print(f'np.dot(Kk, Hk):{np.dot(Kk, Hk)},  np.dot(Kk, Hk),-shape:{np.dot(Kk, Hk).shape}')


xk_p:[[2.5]
 [4. ]], Pk_p:[[0.36 0.5 ]
 [0.5  1.1 ]], Hk:[[0.01 0.  ]], Hk.T:[[0.01]
 [0.  ]]
xk_p-.shape:(2, 1), Pk_p-.shape:(2, 2), Hk-shape:(1, 2), Hk.T-shape:(2, 1)
xk:[[2.67796293]
 [4.24776676]], xk-shape:(2, 1)

Pk:[[0.35871507 0.49821537]
 [0.49821107 1.09751537]], Pk_p:[[0.36 0.5 ]
 [0.5  1.1 ]],  Pk-shape:(2, 2),  Pk_p-shape:(2, 2)
np.dot(Kk, Hk):[[0.00356926 0.        ]
 [0.00496926 0.        ]],  np.dot(Kk, Hk),-shape:(2, 2)


<ipython-input-1-c876af85fd28>:38: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  wk = np.array([0, Qk])
<ipython-input-1-c876af85fd28>:52: DeprecationWarning: setting an array element with a sequence. This was supported in some cases where the elements are arrays with a single element. For example `np.array([1, np.array([2])], dtype=int)`. In the future this will raise the same ValueError as `np.array([1, [2]], dtype=int)`.
  Hk = np.array([[np.round(meas_model[0], 3), 0]], dtype=float)
